In [ ]:
import keras
keras.__version__

# Nadmierne dopasowanie i zbyt słabe dopasowanie

Zwykle przy zastosowaniu sieci neuronowych widzimy pewien wzorzec — wydajność modelu podczas przetwarzania odłożonego na bok walidacyjnego zbioru danych zawsze po kilku epokach osiągała wartość szczytową, a następnie ulegała degradacji — modele zaczynały ulegać nadmiernemu dopasowaniu do danych treningowych. Do nadmiernego dopasowania może dojść podczas pracy nad dowolnym problemem uczenia maszynowego. Pracując z algorytmami uczenia maszynowego, musimy wiedzieć, jak radzić sobie z tym problemem.

Podstawowym problemem uczenia maszynowego jest kompromis między optymalizacją a uogólnianiem. Optymalizacja jest procesem dostrajania modelu w celu uzyskania najlepszej możliwej wydajności na danych treningowych (jest to proces uczenia, od którego wzięła się nazwa uczenie maszynowe), a uogólnianie (generalizacja) odwołuje się do tego, jak dobrze wytrenowany model sprawdza się podczas przetwarzania danych, których nigdy nie widział. Oczywiście chcemy uzyskać jak najlepszą zdolność modelu do uogólniania, ale nie mamy na to wpływu w sposób bezpośredni, ponieważ model możemy modyfikować tylko na danych treningowych.

Na początku trenowania optymalizacja i uogólnianie są ze sobą skorelowane — im mniejsza strata na danych treningowych, tym mniejsza strata na danych testowych. Gdy taka sytuacja ma miejsce, mamy do czynienia ze zbyt słabym dopasowaniem — model może zostać lepiej dopasowany, ponieważ sieć nie dokonała jeszcze modelowania wszystkich wzorców znajdujących się w danych treningowych, ale po pewnej liczbie iteracji algorytmu przetwarzającego dane treningowe uogólnianie przestaje ulegać poprawie, a metryka walidacji przyjmuje wartość stałą lub pogarsza się — wówczas model zaczyna dopasowywać się nadmiernie, a więc zaczyna uczyć się wzorców, które są specyficzne dla danych treningowych i wprowadzają w błąd lub są nieprzydatne podczas przetwarzania nowych danych.

Aby zapobiec uczeniu się przez model błędnych lub zbędnych wzorców treningowego zbioru danych, najlepiej jest zebrać więcej danych treningowych. To dość oczywiste, że model trenowany na większej liczbie obserwacji będzie zdolny do lepszego uogólniania. Jeśli takie rozwiązanie nie jest możliwe, możemy modulować ilość informacji, które model może przechowywać, lub dodać ograniczenia co do możliwości przechowywania informacji przez model. Jeżeli sieć może zapamiętać tylko niewielką liczbę wzorców, to proces optymalizacji wymusi skupienie się na najważniejszych wzorcach, które najprawdopodobniej lepiej sprawdzą się przy uogólnianiu.

Proces walki z nadmiernym dopasowaniem określamy mianem regularyzacji. Chciałbym opisać najpopularniejsze techniki regularyzacji i zastosować je w praktyce w celu poprawy działania modelu klasyfikacji sentymentu (pozytywny / negatywny) na podstawie opinii o filmach z bazy imdb.

Uwaga: W charakterze zbioru walidacyjnego będziemy korzystać ze zbioru testowego IMDB. W tym przypadku nie jest to żadnym problemem.

Przygotujmy dane do analizy:

In [ ]:
from keras.datasets import imdb
import numpy as np

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
EPOCHS = 20

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Zbiór treningowy w postaci wektora.
x_train = vectorize_sequences(train_data)
# Zbiór testowy w postaci wektora.
x_test = vectorize_sequences(test_data)
# Etykiety w postaci wektorów.
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# Zmniejszanie nadmiernego dopasowania

## Redukcja rozmiaru sieci


Najprostszym sposobem zapobiegania powstawaniu nadmiernego dopasowania jest zmniejszenie rozmiaru modelu: zmniejszenie liczby uczonych parametrów, na którą wpływa liczba warstw i liczba jednostek je tworzących. W uczeniu głębokim uczone parametry modelu często określa się mianem pojemności modelu. Model dysponujący większą liczbą parametrów charakteryzuje się większą pojemnością pamięci, a więc może łatwiej uczyć się doskonałego mapowania danych przypominającego swym działaniem słownik. Mapowanie takie nie ma żadnej zdolności uogólniania. Przykładowy model z 500 000 parametrów binarnych mógłby z łatwością nauczyć się klasy każdej cyfry wchodzącej w skład treningowego zbioru danych MNIST: każda z 50 000 cyfr mogłaby zostać opisana przy użyciu zaledwie 10 parametrów binarnych, ale taki model byłby zupełnie nieprzydatny podczas klasyfikacji nowych próbek. Musisz pamiętać o tym, że modele uczenia głębokiego mają tendencję do dopasowywania się do danych treningowych, ale Twoim celem jest osiągnięcie modelu zdolnego do jak najlepszych uogólnień, a nie modelu maksymalnie dopasowanego do danych treningowych.

Jeżeli sieć dysponuje zbyt małą zdolnością zapamiętywania, to nie będzie w stanie tak łatwo nauczyć się bezpośredniego mapowania, a więc w celu minimalizacji strat będzie musiała uczyć się skompresowanych reprezentacji, co pozwoli modelowi nabyć umiejętności przewidywania, a o to nam właśnie chodzi. Jednocześnie należy pamiętać o tym, że modele powinny mieć na tyle dużo parametrów, aby nie ulec zbyt słabemu dopasowaniu — model nie powinien cierpieć z powodu braku możliwości zapamiętywania kolejnych cech. Trzeba znaleźć kompromis między zbyt dużą pojemnością a zbyt małą pojemnością.

Niestety nie ma żadnego magicznego wzoru umożliwiającego określenie właściwej liczby warstw i odpowiednich rozmiarów poszczególnych warstw. W celu znalezienia modelu optymalnego z punktu widzenia analizowanych danych należy sprawdzić działanie zestawu różnych architektur (oczywiście trzeba to robić na zbiorze walidacyjnym, a nie testowym). Szukanie odpowiedniego modelu należy zacząć od niewielkiej liczby warstw i parametrów, a następnie zwiększać rozmiary istniejących warstw i stopniowo dodawać nowe, obserwując spadek wartości straty określanej w procesie walidacji.

Spróbujmy zastosować to rozwiązanie w kontekście sieci klasyfikującej recenzje filmów.

In [ ]:
from keras import models
from keras import layers

#proszę wygenerować sieć o 3 warstwach w kerasie:
# - gęsta z 16 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia
# - gęsta z 16 neuronami, funkcją aktywacji ReLU
# - gęsta z 1 neuronem wyjściowym, funkcja aktywacji sigmoid

benchmark_model = models.Sequential()


In [ ]:
benchmark_model.summary()

In [ ]:
#proszę skompilować model, optimizer RMSprop, funkcja kosztu binary crossentropy i śledzoną miarą accuracy


In [ ]:
#proszę wytrenować powyższy model, liczba epok: EPOCHS, rozmiar batcha: 512, 
#proszę podać dane testowe jako validation_data w procesie uczenia
benchmark_hist = None

In [ ]:
epochs = range(1, EPOCHS+1)
benchmark_val_loss = benchmark_hist.history['val_loss']

Spróbujmy zastąpić ten model prostszą siecią neuronową:

In [ ]:
#proszę wygenerować sieć o 3 warstwach w kerasie z mniejszą liczbą parametrów:
# - gęsta z 4 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia
# - gęsta z 4 neuronami, funkcją aktywacji ReLU
# - gęsta z 1 neuronem wyjściowym, funkcja aktywacji sigmoid
smaller_model = models.Sequential()


In [ ]:
smaller_model.summary()

In [ ]:
#proszę skompilować model, optimizer RMSprop, funkcja kosztu binary crossentropy i śledzoną miarą accuracy



Oto porównanie straty walidacji oryginalnej sieci i mniejszej sieci. Kropkami oznaczono wartości straty walidacji mniejszej sieci, a krzyżykami oznaczono wartości straty oryginalnej sieci (przypominam, że mniejsza wartość straty walidacji świadczy o tym, że model jest lepszy).

In [ ]:
#proszę wytrenować powyższy model, liczba epok: EPOCHS, rozmiar batcha: 512, 
#proszę podać dane testowe jako validation_data w procesie uczenia
smaller_model_hist = None

In [ ]:
smaller_model_val_loss = smaller_model_hist.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt

# b+ to niebieskie krzyżyki
plt.plot(epochs, benchmark_val_loss, 'b+', label='Bazowy model')
# bo to niebieskie kropki
plt.plot(epochs, smaller_model_val_loss, 'bo', label='Mniejszy model')
plt.xlabel('Epoki')
plt.ylabel('Strata walidacji')
plt.legend()

plt.show()


Jak widać, mniejsza sieć zaczęła ulegać nadmiernemu dopasowaniu (przeuczeniu) później niż nasz początkowy model (po sześciu, a nie po czterech epokach), a dodatkowo po przekroczeniu punktu przeuczenia wydajność mniejszego modelu ulega wolniejszej degradacji.

Spróbujmy przeanalizować w tym kontekście działanie sieci o znacznie większej pojemności (przekraczającej potrzeby problemu).

In [ ]:
#proszę wygenerować sieć o 3 warstwach w kerasie z większą liczbą parametrów:
# - gęsta z 512 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia
# - gęsta z 512 neuronami, funkcją aktywacji ReLU
# - gęsta z 1 neuronem wyjściowym, funkcja aktywacji sigmoid
bigger_model = models.Sequential()


In [ ]:
bigger_model.summary()

In [ ]:
#proszę skompilować model, optimizer RMSprop, funkcja kosztu binary crossentropy i śledzoną miarą accuracy


In [ ]:
#proszę wytrenować powyższy model, liczba epok: EPOCHS, rozmiar batcha: 512, 
#proszę podać dane testowe jako validation_data w procesie uczenia
bigger_model_hist = None

Oto rysnek, na którym porównano wydajność zbyt dużej sieci i naszego początkowego modelu. Kropkami oznaczono stratę walidacji większej sieci, a krzyżykami oznaczono stratę walidacji początkowej wersji sieci.

In [ ]:
import matplotlib.pyplot as plt

bigger_model_val_loss = bigger_model_hist.history['val_loss']
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

axes[0].plot(epochs, benchmark_val_loss, 'b+', label='Bazowy model')
axes[0].plot(epochs, bigger_model_val_loss, 'bo', label='Wiekszy model')
axes[0].set_xlabel('Epoki')
axes[0].set_ylabel('Strata walidacji')
axes[0].legend()

axes[1].plot(epochs, benchmark_val_loss, 'b+', label='Bazowy model')
axes[1].plot(epochs, smaller_model_val_loss, 'bo', label='Mniejszy model')
axes[1].set_xlabel('Epoki')
axes[1].set_ylabel('Strata walidacji')
axes[1].legend()

ylim = (0, max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
axes[0].set_ylim(ylim)
axes[1].set_ylim(ylim)

plt.show()


porównano wydajność zbyt dużej sieci i naszego początkowego modelu. Kropkami oznaczono stratę walidacji większej sieci, a krzyżykami oznaczono stratę walidacji początkowej wersji sieci.

Oto rysunek, na którym porównano straty procesu trenowania dwóch sieci.


In [ ]:
import matplotlib.pyplot as plt

benchmark_train_loss = benchmark_hist.history['loss']
smaller_model_train_loss = smaller_model_hist.history['loss']
bigger_model_train_loss = bigger_model_hist.history['loss']


fig, axes = plt.subplots(1, 2, figsize=(15, 4))

axes[0].plot(epochs, benchmark_train_loss, 'b+', label='Bazowy model')
axes[0].plot(epochs, bigger_model_train_loss, 'bo', label='Wiekszy model')
axes[0].set_xlabel('Epoki')
axes[0].set_ylabel('Strata treningowa')
axes[0].legend()

axes[1].plot(epochs, benchmark_train_loss, 'b+', label='Bazowy model')
axes[1].plot(epochs, smaller_model_train_loss, 'bo', label='Mniejszy model')
axes[1].set_xlabel('Epoki')
axes[1].set_ylabel('Strata treningowa')
axes[1].legend()

ylim = (0, max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
axes[0].set_ylim(ylim)
axes[1].set_ylim(ylim)

plt.show()

Widać, że większa sieć bardzo szybko uzyskuje praktycznie zerową wartość straty treningowej. Im większa jest pojemność sieci, tym szybciej modelowane są dane treningowe (uzyskiwana jest niska wartość straty treningowej), ale wzrasta wówczas podatność na nadmierne dopasowanie (powstaje duża różnica między stratą treningową a stratą walidacji).

## Dodawanie regularyzacji wag


Czy znasz zasadę brzytwy Ockhama? Według niej, jeżeli istnieją dwa wyjaśnienia jakiejś teorii, to najprawdopodobniej poprawnym wyjaśnieniem jest to, które jest prostsze — to, które czyni mniej założeń. Zasada ta sprawdza się również w kontekście modeli sieci neuronowych: jeżeli mamy dane treningowe, architekturę sieci i wiele zbiorów wartości wag (wiele modeli) opisujących dane, to prostsze modele są mniej podatne na nadmierne dopasowanie od tych, które są bardziej złożone.

Przyjmijmy, że za prostszy model uważamy model, którego rozkład wartości parametrów charakteryzuje się mniejsza entropią, lub model, który ma mniej parametrów. W związku z tym popularną techniką unikania nadmiernego dopasowania jest wymuszenie na modelu ograniczenia złożoności poprzez przyjmowanie tylko małych wartości wag, co sprawia, że rozkład wartości wag jest bardziej regularny. Zabieg ten określamy mianem regularyzacji wag. Implementuje się go poprzez dodanie do funkcji straty sieci kosztu związanego z dużymi wartościami wag. W praktyce można to zrobić na dwa sposoby:


* Regularyzacja L1 — koszt jest dodawany proporcjonalnie do bezwzględnej wartości współczynników wag (normy L1 wag).

* Regularyzacja L2 — koszt jest dodawany proporcjonalnie do kwadratu wartości współczynników wag (normy L2 wag). W kontekście sieci neuronowych regularyzacja L2 jest również określana mianem rozkładu wag. Pomimo innej nazwy jest to ten sam proces, który w matematyce określamy jako regularyzacja L2.

W pakiecie Keras regularyzację wag dodaje się poprzez przekazanie instancji regularyzatora wagi do warstw sieci za pomocą argumentu w formie słowa kluczowego. Dodajmy regularyzację L2 wag do sieci klasyfikatora recenzji filmów.

In [ ]:
from keras import regularizers

#proszę wygenerować sieć o 3 warstwach w kerasie z regularyzacją L2:
# - gęsta z 16 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia, regularyzacją L2 z argumentem 0.001
# - gęsta z 16 neuronami, funkcją aktywacji ReLU, regularyzacją L2 z argumentem 0.001
# - gęsta z 1 neuronem wyjściowym, funkcja aktywacji sigmoid
l2_model = models.Sequential()


In [ ]:
l2_model.summary()

In [ ]:
#proszę skompilować model, optimizer RMSprop, funkcja kosztu binary crossentropy i śledzoną miarą accuracy


Argument l2(0.001) oznacza, że każdy współczynnik macierzy wag warstwy doda wartość równą 0.001 * weight_coefficient_value (0,001 razy wartość współczynnika wagi) do całkowitej straty sieci. Kara ta jest dodawana tylko podczas trenowania, a więc strata sieci w czasie trenowania będzie o wiele wyższa niż w czasie testowania.

Oto wykres, na którym pokazano wpływ kary w postaci regularyzacji L2:

In [ ]:
#proszę wytrenować powyższy model, liczba epok: EPOCHS, rozmiar batcha: 512, 
#proszę podać dane testowe jako validation_data w procesie uczenia
l2_model_hist = None

In [ ]:
import matplotlib.pyplot as plt

l2_model_val_loss = l2_model_hist.history['val_loss']

plt.plot(epochs, benchmark_val_loss, 'b+', label='Bazowy model')
plt.plot(epochs, l2_model_val_loss, 'bo', label='Model z regularyzacja L2')
plt.xlabel('Epoki')
plt.ylabel('Strata walidacji')
plt.legend()

plt.show()



Jak widać, model z regularyzacją L2 (kropki) stał się o wiele bardziej odporny na nadmierne dopasowanie od modelu referencyjnego (krzyżyki) pomimo tego, że oba modele charakteryzują się identyczną liczbą parametrów.

Zamiast regularyzacji L2 możesz korzystać również z innych mechanizmów regularyzacji obsługiwanych przez pakiet Keras.

In [ ]:
from keras import regularizers

# Regularyzacja L1.
regularizers.l1(0.001)

# Jednoczesna regularyzacja L1 i L2.
regularizers.l1_l2(l1=0.001, l2=0.001)

## Porzucanie — technika dropout


Porzucanie (ang. droput) jest jedną z najbardziej skutecznych i najpopularniejszych technik regularyzacji sieci neuronowych. Opracował ją Geoffrey Hinton podczas współpracy ze swoimi studentami na Uniwersytecie w Toronto. Technika ta polega na losowym wybieraniu pewnej liczby cech wyjściowych warstwy podczas trenowania (wartości tych warstw są zastępowane zerami). Załóżmy, że w czasie trenowania warstwa sieci normalnie zwraca wektor 
>[0.2, 0.5, 1.3, 0.8, 1.1]

Po przeprowadzeniu operacji porzucania w wektorze tym (w losowo wybranych miejscach) pojawią się zera i uzyska on np. formę 
>[0, 0.5, 1.3, 0, 1.1]

Ułamek określający część wyzerowanych cech nazywamy współczynnikiem porzucania (ang. dropout rate). Zwykle parametr ten przyjmuje wartość znajdującą się w przedziale od 0,2 do 0,5. Podczas testowania żadne jednostki nie są porzucane — wartości wyjściowe warstwy sieci są skalowane o współczynnik równy (1 - $\alpha$), gdzie $\alpha$ to współczynnik porzucania. Równoważy to aktywność większej liczby jednostek podczas testowania niż trenowania.



Technika ta może wydawać się dziwna i chaotyczna. Jak ma pomóc w zmniejszeniu nadmiernego dopasowania? Hinton tworząc ją, inspirował się mechanizmami zapobiegającymi nadużyciom stosowanym przez banki. Stwierdził: „Pewnego dnia, gdy poszedłem do banku, zauważyłem, że osoby w okienkach często zmieniają swoje miejsca; pracownicy banku nie potrafili powiedzieć, dlaczego to robią, ale doszedłem do wniosku, że przy takiej rotacji wyłudzenie pieniędzy z banku wymagałoby współpracy wielu pracowników; wówczas zdałem sobie sprawę, że losowe usuwanie różnych podzbiorów neuronów podczas przetwarzania każdego przykładu zapobiegnie konspiracji i zredukuje nadmierne dopasowanie”. Główną ideą tej techniki jest wprowadzenie szumu do wartości wyjściowych warstwy w celu pozbycia się nieznaczących wzorców (Hinton określił je mianem „konspiracji”) — wprowadzenie szumu zapobiega zapamiętywaniu takich wzorców przez sieć.

W pakiecie Keras technikę tę można zastosować przy użyciu warstwy Dropout, którą umieszcza się bezpośrednio za wyjściem znajdującej się wcześniej warstwy:
```bash
model.add(layers.Dropout(0.5))
```

Dodajmy dwie warstwy Dropout do sieci IMDB i zobaczmy, czy pomogą one w zredukowaniu nadmiernego dopasowania:

In [ ]:
#proszę wygenerować sieć o 5 warstwach w kerasie z regularyzacją L2:
# - gęsta z 16 neuronami, funkcją aktywacji ReLU, odpowiednim rozmiarem wejścia
# - dropout ze współczynnikiem odrzucenia 50%
# - gęsta z 16 neuronami, funkcją aktywacji ReLU
# - dropout ze współczynnikiem odrzucenia 50%
# - gęsta z 1 neuronem wyjściowym, funkcja aktywacji sigmoid
dropout_model = models.Sequential()


In [ ]:
dropout_model.summary()

In [ ]:
#proszę skompilować model, optimizer RMSprop, funkcja kosztu binary crossentropy i śledzoną miarą accuracy


In [ ]:
#proszę wytrenować powyższy model, liczba epok: EPOCHS, rozmiar batcha: 512, 
#proszę podać dane testowe jako validation_data w procesie uczenia
dropout_model_hist = None

Czas przedstawić wyniki na wykresie:

In [ ]:
import matplotlib.pyplot as plt
dropout_model_val_loss = dropout_model_hist.history['val_loss']

plt.plot(epochs, benchmark_val_loss, 'b+', label='Bazowy model')
plt.plot(epochs, dropout_model_val_loss, 'bo', label='Model z regularyzacja dropout')
plt.xlabel('Epoki')
plt.ylabel('Strata walidacji')
plt.legend()

plt.show()


Ponownie widać poprawę względem sieci referencyjnej.

Reasumując, oto najczęściej stosowane techniki mające zapobiec nadmiernemu dopasowaniu sieci neuronowych:

* Zdobycie większej ilości danych treningowych.
* Redukcja pojemności sieci.
* Dodanie regularyzacji wag.
* Dodanie mechanizmu porzucania.